In [1]:
from ultralytics import YOLO

In [2]:
import os
import torch
from ultralytics import YOLO

# Resolve OpenMP warning
os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'

def train_yolo_model():
    # Detect and set device
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Training on: {device}")

    # Data path
    data_path = r"D:\Balance\College and Studies\TY\CV & DL Lab\CVDL Dataset\object detection\Persian_Car_Plates_YOLOV8\data.yaml"
    
    # Validate data path
    if not os.path.exists(data_path):
        raise FileNotFoundError(f"Data YAML not found: {data_path}")

    # Initialize and train model
    model = YOLO('yolov8n.pt')
    results = model.train(
        data=data_path,
        epochs=30, #20 must
        imgsz=640,
        batch=16,
        conf=0.5,
        device=device
    )

    return results

if __name__ == '__main__':
    train_yolo_model()

Training on: cpu
New https://pypi.org/project/ultralytics/8.3.36 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.33  Python-3.11.4 torch-2.5.1+cpu CPU (AMD Ryzen 7 5800H with Radeon Graphics)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=D:\Balance\College and Studies\TY\CV & DL Lab\CVDL Dataset\object detection\Persian_Car_Plates_YOLOV8\data.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=0.5, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=

train: Scanning D:\Balance\College and Studies\TY\CV & DL Lab\CVDL Dataset\object detection\Persian_Car_Plates_YOLOV8\t
val: Scanning D:\Balance\College and Studies\TY\CV & DL Lab\CVDL Dataset\object detection\Persian_Car_Plates_YOLOV8\val


Plotting labels to runs\detect\train4\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train4
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1         0G       1.21      3.175       1.19         18        640: 100%|██████████| 14/14 [01:06<00:00,  4.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:07<0

                   all         63         63          0          0          0          0



1 epochs completed in 0.022 hours.
Optimizer stripped from runs\detect\train4\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train4\weights\best.pt, 6.2MB

Validating runs\detect\train4\weights\best.pt...
Ultralytics 8.3.33  Python-3.11.4 torch-2.5.1+cpu CPU (AMD Ryzen 7 5800H with Radeon Graphics)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:05<0

                   all         63         63          0          0          0          0


Speed: 2.5ms preprocess, 82.2ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to runs\detect\train4


In [4]:
import cv2
import math
from ultralytics import YOLO

# Image path
image_path = r"D:\Balance\College and Studies\TY\CV & DL Lab\CVDL Dataset\object detection\Persian_Car_Plates_YOLOV8\extra\test.jpg"

# Load model
model = YOLO(r"runs/detect/train4/weights/best.pt") #runs\detect\train4 only replace
classnames = ['Plates']

# Read image
frame = cv2.imread(image_path)
frame = cv2.resize(frame, (1080,720))

# Detect objects
results = model(frame)

# Process detections
for info in results:
    parameters = info.boxes
    for box in parameters:
        x1, y1, x2, y2 = box.xyxy[0]
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        confidence = box.conf[0]
        class_detect = box.cls[0]
        class_detect = int(class_detect)
        class_detect = classnames[class_detect]
        conf = math.ceil(confidence * 100)
        
        if conf > 50 and class_detect == 'Plates':
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f'{class_detect} {conf}%', (x1, y1-10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

# Display image
cv2.imshow('Detected License Plates', frame)
cv2.waitKey(0)
cv2.destroyAllWindows()


0: 448x640 (no detections), 73.8ms
Speed: 9.0ms preprocess, 73.8ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)
